# Amazon scraping

In [ ]:
# ! pip install requests
# ! pip install beautifulsoup4
# ! pip install matplotlib
# ! pip install ipykernel
# ! pip install requests-html
# ! pip install selenium
# ! pip install unidecode
# ! pip install spacy
# ! pip install wordcloud

In [ ]:
import utils_amazon
from importlib import reload
import os
import pandas as pd
import re
from datetime import datetime
import plotly.express as px

import spacy
from sklearn.feature_extraction.text import CountVectorizer

from IPython.display import display
pd.options.display.max_colwidth = 170
pd.options.display.max_rows = None

In [ ]:
parent_path = r"".replace("\\", "/")

In [ ]:
spacy.cli.download("es_core_news_sm")

In [ ]:
nlp = spacy.load("es_core_news_sm")

In [ ]:
# About 50 mins - 7 pag
reload(utils_amazon)
df = utils_amazon.load_html_search()

In [ ]:
# Look for specif word(s) in comments / Filter

for idx in range(len(df)-1):
    for comm in df.loc[idx,'Global Comments']:
        if re.search(r'calidad', comm, flags=re.IGNORECASE):
            display(df.iloc[idx])

In [ ]:
# Look for specif word(s) in products / Filter

# brand_producs = []
# for i in df[df['Stars']==5]['Product']:
#     if (re.search(r'', i, flags=re.IGNORECASE)) or (re.search(r'', i, flags=re.IGNORECASE)) or (re.search(r'', i, flags=re.IGNORECASE)):
#     # if (re.search(r'', i, flags=re.IGNORECASE)) and (re.search(r'bamb.', i, flags=re.IGNORECASE) or re.search(r'nutre y crece', i, flags=re.IGNORECASE)): # or (re.search(r'[^bamboo].', i, flags=re.IGNORECASE))
#         brand_producs.append(i)


# df_filtered = df[df.Product.isin(brand_producs)].reset_index(drop=True)
# df_filtered = df_filtered.drop_duplicates(subset=['Product', 'Price', 'Stars', 'Global Ratings'], keep='first',ignore_index=True)

# print(df_filtered.shape)
# df_filtered.head()

# ----------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------

# Look for specif word(s) in products / Filter

brand_producs = []
for i in df['Product']:
    if ((re.search(r'', i, flags=re.IGNORECASE)) or (re.search(r'', i, flags=re.IGNORECASE)) or (re.search(r'', i, flags=re.IGNORECASE))) and \
        ((re.search(r'colágeno', i, flags=re.IGNORECASE)) or (re.search(r'menta', i, flags=re.IGNORECASE))):
        brand_producs.append(i)


df_filtered = df[df.Product.isin(brand_producs)].reset_index(drop=True)
df_filtered.drop_duplicates(subset=['Product', 'Price', 'Stars', 'Global Ratings'], keep='first',ignore_index=True, inplace=True)

print(df_filtered.shape)
df_filtered.head()

In [ ]:
# Open a product in browser
utils_amazon.open_link(df,4)

In [ ]:
# Save a filtered DataFrame
df_filtered.to_excel(os.path.join(parent_path,"_{}_{}_{}.xlsx".format(datetime.today().year,datetime.today().month,datetime.today().day)), index=False)

In [ ]:
# Pass the DataFrame and copy and paste the product name or DataFrame idx you want to see in browser
reload(utils_amazon)
utils_amazon.open_link(df,'')

# Word Cloud

In [ ]:
import spacy
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer
from PIL import Image
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
spacy.cli.download("es_core_news_sm")

In [ ]:
nlp = spacy.load("es_core_news_sm")
pat = re.compile(r"[^a-z ]")
spaces = re.compile(r"\s{2,}")

def preprocess(text, min_len=1, max_len=1000):
    # spacy Doc creation
    doc = nlp(text)
    # Remove stopwords
    filtered_tokens = filter(
            lambda token: not token.is_stop,
            doc
            )
    # Filter words by length and remove stop words
    filtered_tokens2 = filter(
            lambda token: len(token) >= min_len and len(token) <= max_len and not token.is_stop,
            filtered_tokens
        )
    # Lemmatization
    lemmas = map(
            lambda token: token.lemma_,
            filtered_tokens2
            )
    lemma_text = " ".join(lemmas)
    # Normalize text
    norm_text = unidecode(lemma_text)
    # Remove accents
    lower_text = norm_text.lower()
    # Remove special characters
    clean_text = re.sub(pat, "", lower_text)
    # Remomove duplicate spaces (if exist)
    spaces_text = re.sub(spaces, " ", clean_text)
    return spaces_text.strip()

In [ ]:
all_comments = []

for comm in df_filtered['Global Comments']:
    all_comments.extend(comm)

print(all_comments)

In [ ]:
prep_comments = list(map(preprocess,all_comments))
print(prep_comments)

In [ ]:
vect = (
    CountVectorizer(max_features=1000, max_df=0.7)
    .fit(prep_comments)
    )

X = vect.transform(prep_comments)
vocab = vect.get_feature_names_out()

counts = np.array(X.sum(axis=0)).flatten()
counts_dict = {word: count for word, count in zip(vocab, counts)}

mask = np.array(Image.open(r"\cloud.png"))

In [ ]:
wc = WordCloud(background_color='white',
        width=3000,
        height=2000,
        collocations=False,
        mask=mask,
        colormap = 'Dark2',
        max_words=40
).generate_from_frequencies(counts_dict)

plt.figure(figsize=[7,7])
plt.imshow(wc,interpolation="bilinear")
plt.axis("off") 
plt.show()

# Competition

In [ ]:
os.listdir(parent_path)

In [ ]:
# Competition: competition brands scraping and then concatenate DataFrames.

comp = ["L'oréal", "OGX", "Tío Nacho"]
files = os.listdir(parent_path)[1::]
df_competition = pd.DataFrame()

# for c in comp:
for f, c in zip(files, comp):
    df_temp = pd.read_excel(os.path.join(parent_path,f))
    df_temp["Brand"] = c
    df_temp.drop_duplicates(subset=['Product', 'Price', 'Stars', 'Ratings'], keep='first',ignore_index=True, inplace=True)
    df_competition = pd.concat([df_competition, df_temp], axis=0).reset_index(drop=True)
    # print(df_temp.shape)
    del df_temp
    
df_competition = df_competition[['Brand', 'Product', 'Price', 'Stars', 'Ratings', 'Ratings MX', 'Comments Global', 'Link']]
print(df_competition.shape)
display(df_competition.head())

In [ ]:
# Save a DataFrame
df_competition.to_excel(os.path.join(parent_path,"competition_{}_{}_{}.xlsx".format(datetime.today().year,datetime.today().month,datetime.today().day)), index=False)

In [ ]:
total_comments = 0
for i in df_competition.loc[(df_competition["Brand"] == "Tío Nacho")].drop_duplicates(subset=['Product', 'Price', 'Stars', 'Ratings'], keep='first',ignore_index=False, inplace=False).sort_values(by='Stars', ascending=False)['Comments Global']:
    total_comments += i.__len__()
print(df_competition.loc[(df_competition["Brand"] == "Tío Nacho")].drop_duplicates(subset=['Product', 'Price', 'Stars', 'Ratings'], keep='first',ignore_index=False, inplace=False).sort_values(by='Stars', ascending=False)['Ratings'].sum())
print(total_comments)

In [ ]:
df_competition.loc[(df_competition["Brand"] == "Tío Nacho")].drop_duplicates(subset=['Product', 'Price', 'Stars', 'Ratings'], keep='first',ignore_index=False, inplace=False).sort_values(by='Stars', ascending=False)

In [ ]:
df_competition.groupby("Brand")["Product"].count()

In [ ]:
# Open a product in browser
utils_amazon.open_link(df_competition,32)

# Metrics

## Price Analysis

In [ ]:
df = pd.read_excel(os.path.join(parent_path,"".replace("\\", "/")))
print(df.shape)
df.head()

In [ ]:
reload(utils_amazon)
df_25 = utils_amazon.quartile_prices(df)[0]
df_25_50 = utils_amazon.quartile_prices(df)[1]
df_50_75 = utils_amazon.quartile_prices(df)[2]
df_75 = utils_amazon.quartile_prices(df)[3]
quartiles = utils_amazon.quartile_prices(df)[4]
quartiles

In [ ]:
# Products above quartile 75
print('Prime products [above quartile 75] -> {} products ({}%) '.format(df_75.Product.to_list().__len__(), round((df_75.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Premium products [between quartiles 50 & 75] -> {} products ({}%) '.format(df_50_75.Product.to_list().__len__(), round((df_50_75.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Mid products [between quartiles 25 & 50] -> {} products ({}%) '.format(df_25_50.Product.to_list().__len__(), round((df_25_50.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Low products [below quartile 25 ] -> {} products ({}%) '.format(df_25.Product.to_list().__len__(), round((df_25.Product.to_list().__len__() / df.__len__()) * 100, 2)))

In [ ]:
print('Quartile (75>) - Products: {}'.format(df_75.__len__()))
display(df_75.head())
print('Quartile (50-75] - Products: {}'.format(df_50_75.__len__()))
display(df_50_75.head())
print('Quartile (25-50] - Products: {}'.format(df_25_50.__len__()))
display(df_25_50.head())
print('Quartile (<25] - Products: {}'.format(df_25.__len__()))
display(df_25.head())

In [ ]:
print(df[df.Price <= 360].shape[0])
px.box(df,'Price', title=f'Price distribution')

In [ ]:
df[df.Price >= 176].shape[0] / df.shape[0]

In [ ]:
display(px.histogram(df, x='Price', title=f'Price distribution'))
display(px.histogram(df_75, x='Price', title=f'Price distribution - Prime Products', nbins=10, text_auto=True))
display(df_75.describe().round(2))
display(px.histogram(df_50_75, x='Price', title=f'Price distribution - Premium Products', nbins=15, text_auto=True))
display(df_50_75.describe().round(2))
display(px.histogram(df_25_50, x='Price', title=f'Price distribution - Mid Products', nbins=15, text_auto=True))
display(df_25_50.describe().round(2))

In [ ]:
display(df_75.Product.unique().tolist()[:10])
print()
display(df_50_75.Product.unique().tolist()[:10])
print()
display(df_25_50.Product.unique().tolist()[:10])

In [ ]:
#Prime Products
display(df_75.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10))
for i in df_75.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10).Product.tolist():
    print(i)

#Premium Products
display(df_50_75.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10))
for i in df_50_75.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10).Product.tolist():
    print(i)

#Mid Products
display(df_25_50.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10))
for i in df_25_50.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10).Product.tolist():
    print(i)

## Brands distribution

In [ ]:
# Look for specif word(s)

brand_producs = []
for i in df['Product']:
    if (re.search(r'colágeno.', i, flags=re.IGNORECASE) or re.search(r'nutre y revitaliza', i, flags=re.IGNORECASE) or re.search(r'nutre & revitaliza', i, flags=re.IGNORECASE)) and (re.search(r'pantene', i, flags=re.IGNORECASE)) and not (re.search(r'bamb', i, flags=re.IGNORECASE) or re.search(r'nutrient blend.', i, flags=re.IGNORECASE)) : # or (re.search(r'[^bamboo].', i, flags=re.IGNORECASE))
        brand_producs.append(i)


brand_df = df[df.Product.isin(brand_producs)].reset_index(drop=True)

#brand_df = brand_df.drop([4,5,7,8,9,10,11,13]).reset_index(drop=True)

brand_df.to_excel(os.path.join(parent_path,"_{}_{}_{}.xlsx".format(datetime.today().year,datetime.today().month,datetime.today().day)), index=False)
print(brand_df.shape)
brand_df

In [ ]:
utils_amazon.open_link(brand_df,8)